# Abstract

Voltage-sensitive dye imaging (VSDi) has revealed fundamental properties of neocortical processing at macroscopic scales. Since for each pixel VSDi signals report the average membrane potential over hundreds of neurons, it seems natural to use a mean-field formalism to model such signals. Here, we present a mean-field model of networks of Adaptive Exponential (AdEx) integrate-and-fire neurons, with conductance-based synaptic interactions. We study a network of regular-spiking (RS) excitatory neurons and fast-spiking (FS) inhibitory neurons. We use a Master Equation formalism, together with a semi-analytic approach to the transfer function of AdEx neurons to describe the average dynamics of the coupled populations. We compare the predictions of this mean-field model to simulated networks of RS-FS cells, first at the level of the spontaneous activity of the network, which is well predicted by the analytical description. Second, we investigate the response of the network to time-varying external input, and show that the mean-field model predicts the response time course of the population. Finally, to model VSDi signals, we consider a one-dimensional ring model made of interconnected RS-FS mean-field units. We found that this model can reproduce the spatio-temporal patterns seen in VSDi of awake monkey visual cortex as a response to local and transient visual stimuli. Conversely, we show that the model allows one to infer physiological parameters from the experimentally-recorded spatio-temporal patterns.

In [6]:
"""
Empty notebook, proper version will be uploaded after acceptance
"""
%matplotlib inline

# the code is in the "modeling_mesoscopic_dynamics" folder
import modeling_mesoscopic_dynamics

### Transfer Functions of Single Cell Models (within a given network setting)

Here, we build up the description of the single cell computation within the mean-field formalism

#### Excitatory cells: RS cells

In [2]:
# the parameters are written within a library of neuron parameters
from modeling_mesoscopic_dynamics.single_cell_models.cell_library import get_neuron_params
RS_params = get_neuron_params('RS-cell', name='RS-cell', SI_units=False)
print(RS_params)

cell parameters --NOT-- in SI units
{'delta_v': 2.0, 'name': 'RS-cell', 'Vreset': -65.0, 'b': 20.0, 'N': 1, 'Vthre': -50.0, 'Cm': 200.0, 'Gl': 10.0, 'a': 4.0, 'El': -65.0, 'tauw': 500.0, 'Trefrac': 5.0}


In [3]:
# plotting the responses to current steps
from modeling_mesoscopic_dynamics.single_cell_models.step_response import make_model_figure
make_model_figure('RS-cell', I0=200e-12);

cell parameters in SI units


In [10]:
# Now making the numerical simulations at various levels of both excitatory and inhibitory inputs
# note the rescaling of the excitatory input to insure that the response remains in the <30 range: it focuses on low range when inhibition is low and on a larger range when inhibition is larger
SEED, tstop, discret_Fe, discret_Fi = 3, 2, 3, 3 # for testing
SEED, tstop, discret_Fe, discret_Fi = 4, 10, 10, 10 # paper's value
!(python modeling_mesoscopic_dynamics/transfer_functions/tf_simulation.py RS-cell CONFIG1 -s --SEED $SEED --tstop $tstop --discret_Fe $discret_Fe  --discret_Fi $discret_Fi --max_Fe 30)

numerical TF data saved in : data/RS-cell_CONFIG1.npy


cell parameters in SI units
synaptic network parameters in SI units


In [1]:
from modeling_mesoscopic_dynamics.transfer_functions.theoretical_tools import make_fit_from_data
P_RS = make_fit_from_data('data/RS-cell_CONFIG1.npy', with_square_terms=True, verbose=False)


Optimization terminated successfully.
         Current function value: 0.041754
         Iterations: 1087
         Function evaluations: 1541
[-54.7150805    7.64453782  -4.62722888   1.71283147  -0.10710902
   0.72054481  -2.00463685   0.11756368   5.61325855  -0.99436164
  -2.87604362] mV
coefficients saved in  data/RS-cell_CONFIG1_fit.npy


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 7.33631085882e-08
            Iterations: 9
            Function evaluations: 120
            Gradient evaluations: 9

In [2]:
# now displaying the numerical values and the analytical function
from modeling_mesoscopic_dynamics.transfer_functions.plots import make_exc_inh_fig
make_exc_inh_fig('data/RS-cell_CONFIG1.npy', P=P_RS)

#### Inhibitory cells: FS-cells

In [3]:
# the parameters are written within a library of neuron parameters
from modeling_mesoscopic_dynamics.single_cell_models.cell_library import get_neuron_params
FS_params = get_neuron_params('FS-cell', name='FS-cell', SI_units=False)
print(FS_params)

cell parameters --NOT-- in SI units
{'Vthre': -50.0, 'a': 0.0, 'El': -65.0, 'Trefrac': 5.0, 'name': 'FS-cell', 'Vreset': -65.0, 'Gl': 10.0, 'N': 1, 'delta_v': 0.5, 'tauw': 1000000000.0, 'b': 0.0, 'Cm': 200.0}


In [3]:
# plotting the responses to current steps
from modeling_mesoscopic_dynamics.single_cell_models.step_response import make_model_figure
make_model_figure('FS-cell', I0=200e-12);

cell parameters in SI units


In [ ]:
# Now making the numerical simulations at various levels of both excitatory and inhibitory inputs
# note the rescaling of the excitatory input to insure that the response remains in the <30 range: it focuses on low range when inhibition is low and on a larger range when inhibition is larger
SEED, tstop, discret_Fe, discret_Fi = 3, 2, 3, 3 # for testing
SEED, tstop, discret_Fe, discret_Fi = 4, 10, 10, 10 # paper's value
!(python modeling_mesoscopic_dynamics/transfer_functions/tf_simulation.py FS-cell CONFIG1 -s --SEED $SEED --tstop $tstop --discret_Fe $discret_Fe  --discret_Fi $discret_Fi --max_Fe 30)

In [5]:
from modeling_mesoscopic_dynamics.transfer_functions.theoretical_tools import make_fit_from_data
P_FS = make_fit_from_data('data/FS-cell_CONFIG1.npy', with_square_terms=True, verbose=False)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 7.65356746695e-08
            Iterations: 7
            Function evaluations: 92
            Gradient evaluations: 7
Optimization terminated successfully.
         Current function value: 0.190404
         Iterations: 731
         Function evaluations: 1043
[ -5.46160664e+01   4.58415750e+00  -1.77303201e+00   6.64785219e-01
  -3.00637490e-01   3.93520293e-01  -5.14454957e-01  -6.39186948e-03
  -1.39021341e+00  -4.85663596e-01  -3.63617754e-01] mV
coefficients saved in  data/FS-cell_CONFIG1_fit.npy


In [6]:
# now displaying the numerical values and the analytical function
from modeling_mesoscopic_dynamics.transfer_functions.plots import make_exc_inh_fig
make_exc_inh_fig('data/FS-cell_CONFIG1.npy', P=P_FS)

In [6]:
# Note that we also used the network parameters because it requires the synaptic properties (number of synapses)
# they are stored in the following module
from modeling_mesoscopic_dynamics.synapses_and_connectivity.syn_and_connec_library import get_connectivity_and_synapses_matrix
print(get_connectivity_and_synapses_matrix('CONFIG1'))

synaptic network parameters --NOT-- in SI units
[[ {'name': 'ee', 'ext_drive': 4.0, 'Ntot': 10000, 'Q': 1.0, 'p_conn': 0.05, 'Erev': 0.0, 'gei': 0.2, 'afferent_exc_fraction': 1.0, 'Tsyn': 5.0}
  {'name': 'ei', 'Tsyn': 5.0, 'Q': 1.0, 'p_conn': 0.05, 'Erev': 0.0}]
 [{'name': 'ie', 'Tsyn': 5.0, 'Q': 5.0, 'p_conn': 0.05, 'Erev': -80.0}
  {'name': 'ii', 'Tsyn': 5.0, 'Q': 5.0, 'p_conn': 0.05, 'Erev': -80.0}]]


### Network dynamics: numerical simulations vs mean-field

performing the numerical simulation with the Brian2 package !
see http://brian2.readthedocs.io/

In [9]:
!(python modeling_mesoscopic_dynamics/network_simulations/ntwk_sim_demo.py --CONFIG RS-cell--FS-cell--CONFIG1 -f data/config1.py)

INFO       No numerical integration method specified for group 'neurongroup', using method 'euler' (took 0.03s, trying other methods took 0.17s). [brian2.stateupdaters.base.method_choice]


INFO       No numerical integration method specified for group 'neurongroup_1', using method 'euler' (took 0.06s, trying other methods took 0.18s). [brian2.stateupdaters.base.method_choice]



        dV/dt = (10.000000*nS*(-65.000000*mV - V) + 10.000000*nS*2.000000*mV*exp(-(-50.000000*mV-V)/(2.000000*mV)) + I - w_adapt)/(200.000000*pF) : volt (unless refractory) 
        dw_adapt/dt = ( -4.000000*nS*( -65.000000*mV - V) - w_adapt )/(500.000000*ms) : amp  
        I = I0 +Gee*(0.000000*mV - V)+Gie*(-80.000000*mV - V) : amp
        dGee/dt = -Gee*(1./(5.000000*ms)) : siemens
        dGie/dt = -Gie*(1./(5.000000*ms)) : siemens
        I0 : amp 
cell parameters --NOT-- in SI units

        dV/dt = (10.000000*nS*(-65.000000*mV - V) + 10.000000*nS*0.500000*mV*exp(-(-50.000000*mV-V)/(0.500000*mV)) + I - w_adapt)/(200.000000*pF) : volt (unless refractory) 
        dw_adapt/dt = ( -0.000000*nS*( -65.000000*mV - V) - w_adapt )/(1000000000.000000*ms) : amp  
        I = I0 +Gei*(0.000000*mV - V)+Gii*(-80.000000*mV - V) : amp
        dGei/dt = -Gei*(1./(5.000000*ms)) : siemens
        dGii/dt = -Gii*(1./(5.000000*ms)) : siemens
        I0 : amp 


synaptic network parameters --NOT-- in SI units
cell parameters --NOT-- in SI units


In [1]:
# showing the ouput of the simulations
%matplotlib inline
from modeling_mesoscopic_dynamics.network_simulations.plot_single_sim import plot_ntwk_sim_output
DATA = np.load('data/config1.npy')
%matplotlib inline
AX, FIG = plot_ntwk_sim_output(*DATA,\
                               zoom_conditions=[0,1000], bar_ms=100,\
                               raster_number=500)

In [3]:
# now comparing the simulation with the mean-field
from modeling_mesoscopic_dynamics.network_simulations.compare_with_mean_field import plot_ntwk_sim_output

FIG = plot_ntwk_sim_output(*DATA,\
                           min_time=500)



Make sure that those two values are similar !!
[ 2.42134221  9.62763885  0.09698788  0.11677506  0.25238966]
[ 2.42134221  9.62763885  0.09698788  0.11677506  0.25238966]
[ 2.42134221  9.62763885  0.09698788  0.11677506  0.25238966]
first order prediction:  0.252389662129
end fixed point


cell parameters in SI units
synaptic network parameters in SI units
starting fixed point
synaptic network parameters in SI units
cell parameters in SI units
cell parameters in SI units
first order prediction:  [ 2.41797829  9.6031657 ]
synaptic network parameters in SI units
cell parameters in SI units
cell parameters in SI units


In [ ]:
S = """
##########################################################################
# we start with the transfer functions
##########################################################################
cd transfer_functions
# FS cell
python tf_simulation.py FS-cell CONFIG1 -s --SEED 4 --tstop 10
python theoretical_tools.py -f data/FS-cell_CONFIG1.npy --With_Square
# RS cell
python tf_simulation.py RS-cell CONFIG1 -s --SEED 4 --tstop 10
python theoretical_tools.py -f data/RS-cell_CONFIG1.npy --With_Square
cd ..
# now network simulations
cd network_simulations
python ntwk_sim_demo.py --CONFIG RS-cell--FS-cell--CONFIG1 -f data/config1.py
# now mean field prediction, passing the output of ntwk sim for comparison
python compare_with_mean_field.py RS-cell--FS-cell--CONFIG1 -f data/config1.py
cd ..
"""    
txt_file = open('generating_all_data.sh', 'w') # writing a baseline file
txt_file.write(S)
txt_file.close()

# Title

Abstract [...]